In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa import arima

In [9]:
data = pd.read_csv('df_merged_weekly_all.csv', index_col=0)
data['date'] = pd.to_datetime(data['date'])
data.reset_index(inplace=True)
data

,level_0,index,week,date,cases,temperature,pressure,humidity,precipitation
0,0,0,2016-1,2016-01-10,19.0,303.437143,1012.285714,71.428571,1.02
1,1,1,2016-2,2016-01-17,23.0,303.908571,1012.857143,65.571429,0.00
2,2,2,2016-3,2016-01-24,31.0,302.251429,1014.428571,76.857143,32.85
3,3,3,2016-4,2016-01-31,26.0,302.778571,1011.142857,75.142857,17.05
4,4,4,2016-5,2016-02-07,10.0,303.285714,1012.428571,70.571429,3.70
...,...,...,...,...,...,...,...,...,...
393,393,135,2024-32,2024-08-05,226.0,302.210000,1009.000000,75.000000,8.78
394,394,136,2024-33,2024-08-12,205.0,303.050000,1007.000000,70.000000,4.91
395,395,137,2024-34,2024-08-19,209.0,303.620000,1009.000000,67.000000,11.63
396,396,138,2024-35,2024-08-26,162.0,302.490000,1008.000000,73.000000,10.55


In [7]:
import warnings
warnings.filterwarnings("ignore")
from statsmodels.tsa.arima.model import ARIMA
import pandas as pd
from sklearn.metrics import mean_absolute_error

def train_arima(data, target, date_col, n_ahead):
    # Sort data by date
    data = data.sort_values(by=date_col)
    
    # Train-test split
    train = data[data[date_col].dt.year < 2023]
    test = data[data[date_col].dt.year >= 2023]
    train_y = train[target]
    test_y = test[target]
    
    # Fit ARIMA model
    model = ARIMA(train_y)
    arima_result = model.fit()
    
    # Forecast future values
    forecast = arima_result.forecast(steps=len(test_y))
    
    # Compute mean absolute error (MAE)
    MAE = mean_absolute_error(test_y, forecast)
    
    return forecast, MAE, arima_result.summary()

In [10]:
target = 'cases'
date = 'date'
prediction_1_week, MAE_1_week, rf_model_1_week = train_arima(data, target, date, 1)
prediction_4_week, MAE_4_week, rf_model_4_week = train_arima(data, target, date, 4)
prediction_12_week,MAE_12_week, rf_model_12_week = train_arima(data, target, date, 12)

In [12]:
rf_model_1_week

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                  cases   No. Observations:                  310
Model:                          ARIMA   Log Likelihood               -1619.777
Date:                Wed, 05 Feb 2025   AIC                           3243.554
Time:                        00:49:44   BIC                           3251.027
Sample:                             0   HQIC                          3246.542
                                - 310                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         25.3613      5.137      4.937      0.000      15.294      35.429
sigma2      2022.7633    103.902     19.468      0.000    1819.120    2226.407
===================================================================================
Ljung-Box (L1) (Q):                 274.20   Jarque-Bera (JB):              4849.30
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.60   Skew:                             3.86
Prob(H) (two-sided):                  0.01   Kurtosis:                        20.77
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""